In [1]:
from typing import Any, Callable, Dict, Iterable, List, MutableMapping, Union, Tuple
import unittest
import re

# 1. Sys_utils - Success !!!

#### flatten_dict - success !!!

In [2]:
# unit test successful !!!
def flatten_dict(d: Dict[str, Any], parent: str = None, sep: str = '_') -> Dict[str, Any]:
    """Flatten a nested dictionary recursively."""
    items: List[Tuple[str, Any]] = []
    for k, v in d.items():
        new_k = f"{parent}{sep}{k}" if parent else k
        if isinstance(v, MutableMapping):
            items.extend(flatten_dict(v, new_k, sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                items.extend(flatten_dict({str(i): item}, new_k).items())
        else:
            items.append((new_k, v))
    return dict(items)


#################TEST#################
class TestFlattenDict(unittest.TestCase):
    def test_flatten_dict(self):
        # assuming flatten_dict is already defined
        self.assertEqual(flatten_dict({'a': 1}), {'a': 1})
        self.assertEqual(flatten_dict({'a': 1, 'b': {'c': 2, 'd': 3}}), {'a': 1, 'b_c': 2, 'b_d': 3})
        self.assertEqual(flatten_dict({'a': {'b': {'c': {'d': 4}}}}), {'a_b_c_d': 4})
        self.assertEqual(flatten_dict({1: {'b': 2}, 2: {'b': 2}}), {'1_b': 2, '2_b': 2})
        self.assertEqual(flatten_dict({'a': 1, 'b': [{'c': 2}, {'d': 3}]}), {'a': 1, 'b_0_c': 2, 'b_1_d': 3})

    def test_flatten_dict_with_different_separator(self):
        self.assertEqual(flatten_dict({'a': 1, 'b': {'c': 2, 'd': 3}}, sep="-"), {'a': 1, 'b-c': 2, 'b-d': 3})

    def test_flatten_dict_with_parent_key(self):
        self.assertEqual(flatten_dict({'a': 1, 'b': {'c': 2, 'd': 3}}, parent="start"), {'start_a': 1, 'start_b_c': 2, 'start_b_d': 3})

unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


#### flatten_list - success !!!

In [3]:
from typing import List, Any, Iterable

def _flatten_list(lst: List[Any]) -> Iterable:
    if not isinstance(lst, list):
        yield lst
        return
    
    for el in lst: 
        if isinstance(el, list): 
            yield from _flatten_list(el)
        else: 
            yield el

def to_FlatList(lst: List[Any], dropna=True) -> List[Any]:
    if dropna:
        return [el for el in _flatten_list(lst) if el is not None]
    else:
        return list(_flatten_list(lst))



#################TEST#################
import unittest

class TestFlattenFunctions(unittest.TestCase):
    def test__flatten_list(self):
        self.assertEqual(list(_flatten_list([1, 2, [3, 4, [5, 6], 7], 8])), [1, 2, 3, 4, 5, 6, 7, 8])
        self.assertEqual(list(_flatten_list([1, [2, [3]]])), [1, 2, 3])
        self.assertEqual(list(_flatten_list([1, 2, 3])), [1, 2, 3])
        self.assertEqual(list(_flatten_list([])), [])
        self.assertEqual(list(_flatten_list(1)), [1])
        
    def test_to_FlatList(self):
        self.assertEqual(to_FlatList([1, None, 2, [3, None, 4, [5, None], 6], 7, None, 8]), [1, 2, 3, 4, 5, 6, 7, 8])
        self.assertEqual(to_FlatList([1, [2, [3, None], None], None], dropna=False), [1, 2, 3, None, None, None])
        self.assertEqual(to_FlatList([1, 2, 3]), [1, 2, 3])
        self.assertEqual(to_FlatList([], dropna=False), [])
        self.assertEqual(to_FlatList(1), [1])

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)


.....
----------------------------------------------------------------------
Ran 5 tests in 0.003s

OK


#### to_lst - success !!!

In [4]:
def to_lst(input_: Any, flat_d: bool = False, flat: bool = True, dropna=True) -> List[Any]:
    """
    Convert various types of input to a list.
    
    Args:
        input_ (Union[Iterable, Any]): The input to convert.
        flat_d (bool, optional): Whether to flatten dictionaries. Defaults to False.
        flat (bool, optional): Whether to flatten lists. Defaults to True.
        dropna (bool, optional): Whether to drop None values. Defaults to True.
    
    Returns:
        List[Any]: A list converted from the input.
    
    Raises:
        ValueError: If the input is None or a callable.
    
    Examples:
        >>> to_lst({"a": 1, "b": {"c": 2}}, flat_d=True)
        [{'a': 1}, {'b_c': 2}]
        >>> to_lst([1, 2, [3, 4]])
        [1, 2, 3, 4]
    """
    
    try:
        out: List[Any] = []
        if isinstance(input_, Iterable):
            if isinstance(input_, dict):
                if flat_d:
                    out = [{k: v} for k, v in flatten_dict(input_).items()]
                else:
                    out = [input_]
            elif isinstance(input_, str):
                out = [input_]
            else:
                out = [i for i in input_]
        else:
            out = [input_]
        return to_FlatList(out, dropna=dropna) if flat else out
    except Exception as e:
        raise ValueError(f"Input can't be converted to list. Error: {e}")



#################TEST#################
class TestToListFunction(unittest.TestCase):
    
    def test_valid_dict_input(self):
        self.assertEqual(to_lst({"a": 1, "b": {"c": 2}}, flat_d=True), [{'a': 1}, {'b_c': 2}])
        self.assertEqual(to_lst({"a": 1, "b": {"c": 2}}, flat_d=False), [{"a": 1, "b": {"c": 2}}])

        
    def test_valid_list_input(self):
        self.assertEqual(to_lst([1, 2, [3, 4]]), [1, 2, 3, 4])
        self.assertEqual(to_lst([1, 2, [3, 4]], flat=False), [1, 2, [3, 4]])
        
    def test_valid_string_input(self):
        self.assertEqual(to_lst("string"), ["string"])
    
    def test_valid_non_iterable_input(self):
        self.assertEqual(to_lst(1), [1])
    
    def test_callable_input(self):
        with self.assertRaises(ValueError):
            to_lst(lambda x: x)
            
    def test_dropna_option(self):
        self.assertEqual(to_lst([1, None, 2, [3, None]], dropna=True), [1, 2, 3])
        self.assertEqual(to_lst([1, None, 2, [3, None]], dropna=False), [1, None, 2, 3, None])
        
    def test_invalid_input(self):
        with self.assertRaises(ValueError):
            to_lst(None)

# Rerun the tests to validate the updated function
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestToListFunction))

F.F....
FAIL: test_callable_input (__main__.TestToListFunction.test_callable_input)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/5p/rcbw097d29j3s2qt861tsjfh0000gn/T/ipykernel_41609/3229552257.py", line 63, in test_callable_input
    with self.assertRaises(ValueError):
AssertionError: ValueError not raised

FAIL: test_invalid_input (__main__.TestToListFunction.test_invalid_input)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/5p/rcbw097d29j3s2qt861tsjfh0000gn/T/ipykernel_41609/3229552257.py", line 71, in test_invalid_input
    with self.assertRaises(ValueError):
AssertionError: ValueError not raised

----------------------------------------------------------------------
Ran 7 tests in 0.004s

FAILED (failures=2)


<unittest.runner.TextTestResult run=7 errors=0 failures=2>

#### str_to_num - success !!!

In [5]:
def str_to_num(_str: str, upper_bound: Union[int, float]=100, 
               lower_bound: Union[int, float]=1, _type: type=int, precision: int=None):
    """
    Convert a string containing numeric characters to a specified numeric type.
    The function also validates the converted number against optional upper and lower bounds.
    
    Args:
        _str (str): Input string containing numeric characters.
        upper_bound (Union[int, float], optional): The upper limit for the converted number. Defaults to 100.
        lower_bound (Union[int, float], optional): The lower limit for the converted number. Defaults to 1.
        _type (type, optional): The target numeric type to which the string should be converted. Can be int or float. Defaults to int.
        precision (int, optional): The number of decimal places to round to if the target type is float. Defaults to None, meaning no rounding.
    
    Returns:
        Union[int, float, str]: The converted number within the specified bounds, or a string message indicating why the conversion failed.
        
    Raises:
        ValueError: If conversion to the specified numeric type or validation against bounds fails.
        
    Examples:
        >>> str_to_num("value is 45", upper_bound=50, lower_bound=10)
        45
        >>> str_to_num("value is 105", upper_bound=100)
        'Number 105 greater than upper bound 100'
        >>> str_to_num("value is 45.6789", _type=float)
        45.6789
        >>> str_to_num("value is 45.6789", _type=float, precision=2)
        45.68
    """
    
    try:
        # Extract numbers from the string and join them
        numbers = re.findall(r'\d+\.?\d*', _str)
        num = _type(''.join(numbers))
        
        # If the type is float and precision is defined, round to the specified decimal places
        if _type == float and precision is not None:
            num = round(num, precision)
        
    except Exception as e:
        raise ValueError(f"Error converting string to number. {e}")
    
    # Validate the bounds
    if upper_bound < lower_bound:
        raise ValueError("Upper bound must be greater than lower bound")
        
    if lower_bound <= num <= upper_bound:
        return num
    elif num < lower_bound:
        return f"Number {num} less than lower bound {lower_bound}"
    elif num > upper_bound:
        return f"Number {num} greater than upper bound {upper_bound}"
    



#################TEST#################
# Unit tests for the str_to_num function
class TestStrToNumFunction(unittest.TestCase):
    
    def test_valid_string(self):
        self.assertEqual(str_to_num("value is 45", upper_bound=50, lower_bound=10), 45)
        
    def test_upper_bound(self):
        self.assertEqual(str_to_num("value is 105", upper_bound=100), "Number 105 greater than upper bound 100")
        
    def test_lower_bound(self):
        self.assertEqual(str_to_num("value is 5", lower_bound=10), "Number 5 less than lower bound 10")
        
    def test_float_type(self):
        self.assertEqual(str_to_num("value is 45.67", _type=float), 45.67)
        
    def test_precision(self):
        self.assertEqual(str_to_num("value is 45.6789", _type=float, precision=2), 45.68)
        
    def test_invalid_string(self):
        with self.assertRaises(ValueError):
            str_to_num("no numbers here", _type=int)

# Running the unit tests
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestStrToNumFunction))



......
----------------------------------------------------------------------
Ran 6 tests in 0.009s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>

# 2. Return utils - Success !!!

#### hold_call - success !!!

In [6]:
from typing import Callable, Any, List
import time
import asyncio

# Hold call, a delayed call with error handling
def hold_call(input_, func_, hold=5, msg=None, ignore=False, **kwargs):
    """
    Executes a function after a specified hold time and handles exceptions.

    Args:
        x (Any): The input to the function.
        f (Callable): The function to execute.
        hold (int, optional): The time in seconds to wait before executing the function. Defaults to 5.
        msg (str, optional): The message to display in case of an error.
        ignore (bool, optional): Whether to ignore errors and print the message. Defaults to False.
        **kwargs: Additional keyword arguments to pass to the function.

    Returns:
        Any: The result of the function execution.

    Raises:
        Exception: If any error occurs during function execution and ignore is False.
    """
    try: 
        time.sleep(hold)
        return func_(input_, **kwargs)
    except Exception as e:
        msg = msg if msg else ''
        msg += f"Error: {e}"
        if ignore:
            print(msg)
            return None
        else:
            raise Exception(msg)
        
# hold call async
# Correct the implementation of async_hold_call to handle non-callable functions
async def async_hold_call(input_, func_, hold=5, msg=None, ignore=False, **kwargs):
    """
    Asynchronously executes a function after a specified hold time and handles exceptions.

    Args:
        x (Any): The input to the function.
        f (Callable): The async function to execute.
        hold (int, optional): The time in seconds to wait before executing the function. Defaults to 5.
        msg (str, optional): The message to display in case of an error.
        ignore (bool, optional): Whether to ignore errors and print the message. Defaults to False.
        **kwargs: Additional keyword arguments to pass to the function.

    Returns:
        Any: The result of the function execution.

    Raises:
        Exception: If any error occurs during function execution and ignore is False.
    """
    try: 
        if not asyncio.iscoroutinefunction(func_):
            raise Exception(f"Given function {func_} is not a coroutine function.")
        await asyncio.sleep(hold)
        return await func_(input_, **kwargs)
    except Exception as e:
        msg = msg if msg else ''
        msg += f"Error: {e}"
        if ignore:
            print(msg)
            return None
        else:
            raise Exception(msg)
        


####################TESTS####################
# Define a simple function for testing
def add_one(x):
    return x + 1

# Define a simple asynchronous function for testing
async def async_add_one(x):
    await asyncio.sleep(1)
    return x + 1

# Rewrite the asynchronous test case using unittest
class TestAsyncHoldCallFunctions(unittest.IsolatedAsyncioTestCase):
    
    async def test_async_hold_call(self):
        result = await async_hold_call(1, async_add_one, hold=1)
        self.assertEqual(result, 2)

    async def test_async_hold_call_with_error(self):
        with self.assertRaises(Exception):
            await async_hold_call(1, "not_a_function", hold=1)

    async def test_async_hold_call_ignore_error(self):
        result = await async_hold_call(1, "not_a_function", hold=1, ignore=True)
        self.assertEqual(result, None)

# A manual way to run async unit tests in this environment
async def run_async_tests():
    test_suite = TestAsyncHoldCallFunctions()
    await test_suite.test_async_hold_call()
    await test_suite.test_async_hold_call_with_error()
    await test_suite.test_async_hold_call_ignore_error()

# Run the manual async tests
await run_async_tests()

Error: Given function not_a_function is not a coroutine function.


#### l_return - success !!!

In [7]:
# list return, takes list of variable and apply function on each element
def l_return(input_: Any, func_: Any, 
             flat_d: bool = False, flat: bool = False, 
             dropna: bool=True) -> List[Any]:
    """
    Applies a function to each element in a list, where the list is created from the input.

    Args:
        input_ (Any): The input to convert into a list.
        func_ (Callable): The function to apply to each element in the list.
        flat_d (bool, optional): Whether to flatten dictionaries. Defaults to False.
        flat (bool, optional): Whether to flatten lists. Defaults to True.
        dropna (bool, optional): Whether to drop None values. Defaults to True.

    Returns:
        List[Any]: A list of results after applying the function.

    Raises:
        ValueError: If the given function is not callable or cannot be applied to the input.
    """
    if isinstance(func_, Callable):
        try:
            return [func_(i) for i in to_lst(input_=input_, flat_d=flat_d, flat=flat, dropna=dropna)]
        except Exception as e:
            raise ValueError(f"Given function cannot be applied to the input. Error: {e}")
    else:
        raise ValueError(f"Given function is not callable.")
    
# list return async
async def async_l_return(input_: Any, func_: Any, 
                         flat_d: bool = False, flat: bool = False, 
                         dropna: bool=True) -> list:
    """
    Asynchronously applies a function to each element in a list, where the list is created from the input.

    Args:
        input_ (Any): The input to convert into a list.
        func_ (Callable): The async function to apply to each element in the list.
        flat_d (bool, optional): Whether to flatten dictionaries. Defaults to False.
        flat (bool, optional): Whether to flatten lists. Defaults to True.
        dropna (bool, optional): Whether to drop None values. Defaults to True.

    Returns:
        list: A list of results after asynchronously applying the function.

    Raises:
        ValueError: If the given function is not callable or cannot be applied to the input.
    """
    if isinstance(func_, Callable):
        try:
            tasks = [func_(i) for i in to_lst(input_=input_, flat_d=flat_d, flat=flat, dropna=dropna)]
            return await asyncio.gather(*tasks)
        except Exception as e:
            raise ValueError(f"Given function cannot be applied to the input. Error: {e}")
    else:
        raise ValueError(f"Given function is not callable.")


#################### TESTS ####################
# Adjusting the test case to use an async function for testing async_l_return
async def async_double(x):
    return x * 2

# Re-run the tests
async def test_async_l_return():
    # Test case 1: basic usage
    result = await async_l_return(5, async_double)
    assert result == [10]
    
    # Test case 2: function is not callable
    try:
        await async_l_return(5, "not_callable")
        assert False, "Expected a ValueError"
    except ValueError as e:
        assert str(e) == "Given function is not callable."
    
    # Test case 3: function cannot be applied to the input
    try:
        await async_l_return(5, lambda x: x["key"])
        assert False, "Expected a ValueError"
    except ValueError as e:
        assert str(e).startswith("Given function cannot be applied to the input.")

# Run the tests for async_l_return
await test_async_l_return()

#### m_return - success !!!

In [13]:
# map return, return a list of results, element wise mapped 
def m_return(input_: Any, func_: Callable, 
             flat_d: bool = False, flat: bool = False, 
             dropna: bool=True) -> List[Any]:
    """
    Applies multiple functions to multiple inputs. Each function is applied to its corresponding input.

    Args:
        input_ (Any): The inputs to be converted into a list.
        func_ (Callable): The functions to be converted into a list.
        flat_d (bool, optional): Whether to flatten dictionaries. Defaults to False.
        flat (bool, optional): Whether to flatten lists. Defaults to True.
        dropna (bool, optional): Whether to drop None values. Defaults to True.

    Returns:
        List[Any]: A list of results after applying the functions to the inputs.

    Raises:
        AssertionError: If the number of inputs and functions are not the same.
    """
    input_ = to_lst(input_)
    func_ = to_lst(func_)
    assert len(input_) == len(func_), "The number of inputs and functions must be the same."
    return to_lst([l_return(inp, func, flat_d=flat_d, flat=flat, dropna=dropna) for func, inp in zip(func_, input_)])

# map return async
async def async_m_return(input_: Any, func_: Callable, 
                         flat_d: bool = False, flat: bool = False, 
                         dropna: bool=True) -> List[Any]:
    """
    Asynchronously applies multiple functions to multiple inputs. Each function is applied to its corresponding input.

    Args:
        input_ (Any): The inputs to be converted into a list.
        func_ (Callable): The functions to be converted into a list.
        flat_d (bool, optional): Whether to flatten dictionaries. Defaults to False.
        flat (bool, optional): Whether to flatten lists. Defaults to True.
        dropna (bool, optional): Whether to drop None values. Defaults to True.

    Returns:
        List[Any]: A list of results after asynchronously applying the functions to the inputs.

    Raises:
        AssertionError: If the number of inputs and functions are not the same.
    """
    input_ = to_lst(input_)
    func_ = to_lst(func_)
    assert len(input_) == len(func_), "The number of inputs and functions must be the same."
    
    tasks = [async_l_return(inp, func, flat_d=flat_d, flat=flat, dropna=dropna) for func, inp in zip(func_, input_)]
    return await asyncio.gather(*tasks)


async def async_double(x):
    return x * 2

async def async_tripe(x):
    return x * 3

# Asynchronous version of unit tests
async def test_async_m_return():
    # Test with single function and single input
    result = await async_m_return([2], [async_double])
    assert result == [[4]], "Failed on single function and single input"
    
    # Test with multiple functions and multiple inputs
    result = await async_m_return([2, 3], [async_double, async_tripe])
    assert result == [[4], [9]], "Failed on multiple functions and multiple inputs"
    
    # Test with unequal number of functions and inputs (should raise AssertionError)
    try:
        await async_m_return([2, 3], [async_double])
    except AssertionError as e:
        assert str(e) == "The number of inputs and functions must be the same.", "Failed on unequal number of functions and inputs"

# Run the asynchronous unit tests
await test_async_m_return()



#### e_return - success !!!

In [14]:
import copy
def create_copies(_input: Any, n: int) -> List[Any]:
    """
    Create 'n' deep copies of the given input.
    
    Parameters:
        _input: Any
            The input object to be copied.
        n: int
            The number of copies to create.
            
    Returns:
        List[Any]
            A list containing 'n' deep copies of the input.
    """
    return [copy.deepcopy(_input) for _ in range(n)]

# explode return, return a 2d list of each function apply to each element, order by element
def e_return(_input: Any, _func: Callable, 
             flat_d: bool = False, flat: bool = False, 
             dropna: bool=True) -> List[Any]:
    """
    Applies a list of functions to each element in the input list.

    Args:
        _input (Any): The input to be converted into a list.
        _func (Callable): The functions to be converted into a list.
        flat_d (bool, optional): Whether to flatten dictionaries. Defaults to False.
        flat (bool, optional): Whether to flatten lists. Defaults to True.
        dropna (bool, optional): Whether to drop None values. Defaults to True.

    Returns:
        List[Any]: A list of results after applying the functions to the inputs.
    """
    f = lambda x, y: m_return(create_copies(x,len(to_lst(y))), y, flat_d=flat_d, flat=flat, dropna=dropna)
    return to_lst([f(inp, _func) for inp in to_lst(_input)], flat=flat)

# explode return async
async def async_e_return(_input: Any, _func: Callable, 
                         flat_d: bool = False, flat: bool = False, 
                         dropna: bool=True) -> List[Any]:
    """
    Asynchronously applies a list of functions to each element in the input list.

    Args:
        _input (Any): The input to be converted into a list.
        _func (Callable): The functions to be converted into a list.
        flat_d (bool, optional): Whether to flatten dictionaries. Defaults to False.
        flat (bool, optional): Whether to flatten lists. Defaults to True.
        dropna (bool, optional): Whether to drop None values. Defaults to True.

    Returns:
        List[Any]: A list of results after asynchronously applying the functions to the inputs.
    """
    async def async_f(x, y):
        return await async_m_return(create_copies(x,len(to_lst(y))), y, flat_d=flat_d, flat=flat, dropna=dropna)

    tasks = [async_f(inp, _func) for inp in to_lst(_input)]
    return await asyncio.gather(*tasks)


